# GETTING AND LOADING A CORPUS

https://nlpforhackers.io/corpora/
http://lucumr.pocoo.org/2015/11/18/pythons-hidden-re-gems/

The goal of this step is to develop an initial list of each character and their spoken lines. Dictionaries are Hash value arbitrary, so may not be ordered the same.

In [1]:
#Select and Read a file into "f" using a list and stripping out all Project Gutenberg headers and footers
from pathlib import Path
import re
idx = 1
data_folder = Path("data/poe/")
file_to_open = data_folder / "2149-0.txt"
f = open(file_to_open, 'r')
first_document = list(f)
#print(first_document)

first_header_index = 0
second_header_index = 0
footer_index = 0

for first_header_index in range( len(first_document) ):
    if ( ( first_document[first_header_index].find('*END*THE SMALL PRINT!') ) != -1 ) :
        break

for first_header_index in range( len(first_document) ):
    if ( ( first_document[first_header_index].find('START OF THIS PROJECT GUTENBERG') ) != -1 ) :
        break        
        
second_document = list(first_document[first_header_index + 1 :])

for second_header_index in range( len(second_document) ):
    if ( ( second_document[second_header_index].find('www.gutenberg.org') ) != -1 ) :
        break            
for footer_index in range( len(first_document) ):
    if ( ( first_document[footer_index].find('End of Project') ) != -1 ) :
        break
            
print(first_header_index)            
print(second_header_index)
print(footer_index)     
    
script = list()
if (second_header_index < (first_header_index + 100)):
    script = list(first_document[first_header_index +1 + second_header_index +1 : footer_index-1])
else:
    script = list(first_document[first_header_index +1 : footer_index-1])
#print(script)

20
9261
9276


In [2]:
#Compile a list of speakers
r = re.compile("[A-Z0-9][A-Z0-9]+")
speakers = []
for line in script:
    mtch = r.match(line)
    if mtch:
        speakers.append(mtch.group())
print(speakers)

#Omit speakers from the list of text
s = re.compile(r"\b[A-Z{3}\.]+\b")
spoken = list(filter(lambda i: not s.search(i), script))

print(spoken)

['THE', 'IN', 'VOLUME', 'CONTENTS', 'NARRATIVE', 'INTRODUCTORY', 'UPON', 'CHAPTER', 'MY', 'CHAPTER', 'IN', 'CHAPTER', 'THE', 'CHAPTER', 'THE', 'CHAPTER', 'FOR', 'CHAPTER', 'THE', 'CHAPTER', 'JULY', 'CHAPTER', 'AS', 'CHAPTER', 'LUCKILY', 'CHAPTER', 'SHORTLY', 'CHAPTER', 'WE', 'CHAPTER', 'CHAPTER', 'JULY', 'CHAPTER', 'THE', '48', '69', 'CHAPTER', 'ON', '12', 'CHAPTER', '60', '31', '54', '15', '30', 'CHAPTER', '41', '21', '42', 'CHAPTER', 'CHAPTER', 'CHAPTER', 'THE', 'CHAPTER', 'AS', 'CHAPTER', 'OUR', 'CHAPTER', 'DURING', 'CHAPTER', 'ON', 'CHAPTER', 'WE', 'NOTE', 'THE', 'LIGEIA', 'MORELLA', 'WITH', 'DURING', 'THE', 'MANY', 'KING', 'ABOUT', 'THREE', 'CAPT', 'SMITHERTON', 'UNCLE', 'KATE', 'UNCLE', 'KATE', 'UNCLE', 'MYSELF', 'PRATT', 'KATE', 'PRATT', 'SMITHERTON', 'ALL', 'UNCLE', 'PRATT', 'SMITHERTON', 'KATE', 'SMITHERTON', 'UNCLE', 'SMITHERTON', 'UNCLE', 'SMITHERTON', 'UNCLE']
['\n', '\n', '\n', '\n', 'Produced by David Widger\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', 'The Raven E

In [3]:
#Concatenate lines into list entries for future sentence splitting

newLines = []
singleLine = ''
singleLines = []

#Remove all line returns(ok)
for j in range(0, len(spoken)):
    spoken[j] = spoken[j].replace('\n', '')
    
#Split 5 lines at a time into new list
for k in range( 0, len(spoken), 3):
    newLines = []
    for line in range( 0, 3 ):
        try:
            newLines.append(' '+spoken[line+k])
        except:
            print("Index Error at", k, line)
            break
    #Join 5-line groups into one line and append to a list
    singleLine = ''.join(newLines)
    singleLines.append(singleLine)

#print(singleLines)

Index Error at 7596 1


In [4]:
#Create list of sentences
sentences = []
for m in range(0, len(singleLines)):
    mtch = re.findall("[A-Z][^\.!?]*[\.!?]", singleLines[m], re.M|re.I)
    if mtch:
        sentences.append(mtch)
#print(sentences)

In [5]:
#Clean the stopword list
stoplist = []
clean_line = []
data_folder = Path("data/")
file_to_open = data_folder / "snowball_stop.txt"
f = open(file_to_open, 'r')
full_stop = list(f)

for n in range( 0, len(full_stop), 1 ):
    clean_line = full_stop[n].split('|')
    stoplist.append(clean_line[0])

for p in range(len(stoplist)):
    stoplist[p] = stoplist[p].replace('\n', '')
    
#print(stoplist)

In [59]:
#Create list of phrases using stopwords
phrases = []
candidate_phrases = []

for q in range(len(sentences)):
    for r in sentences[q]:
        words = re.split("\\s+", r)
        previous_stop = False
 
        # examine each word to determine if it is a phrase boundary marker or part of a phrase or alone
        for w in words:
 
            if w in stoplist and not previous_stop:
                # phrase boundary encountered, so put a hard indicator
                candidate_phrases.append(";")
                previous_stop = True
            elif w not in stoplist and len(w) > 3:
                # keep adding words to list until a phrase boundary is detected
                candidate_phrases.append(w.strip())
                previous_stop = False
 
    # get a list of candidate phrases without boundary demarcation
    phrases = re.split(";+", ' '.join(candidate_phrases))

# Celan up phrases    
re2 = re.compile('[^\.!?,"(){}\*:]*[\.!?,"(){}\*:]')
for s in range(len(phrases)):
    phrases[s] = re.sub(re2, ' ', phrases[s])
    phrases[s] = phrases[s].strip(' ')
    phrases[s] = phrases[s].replace(' ', '_')
    phrases[s] = phrases[s].replace('__', '_')
    phrases[s] = phrases[s].strip('_')

try:
    phrases.remove('')
    phrases.remove(' ')
    phrases.remove('/n')
    phrases.remove('/n')
except:
    pass
    
for t in range(len(phrases)):
    print(phrases[t])

extraordinary_series
adventure
South_Seas
account_given
accident_threw
society
several_gentlemen
were_constantly_urging_upon

give_narrative
were

concern
person
myself
others
much_unavoidable_exaggeration
prone
detailing_events
have_powerful_influence
exciting

veracity--the_probability_being
public
large
principal_causes
prevented
complying
suggestions

related
lately_editor
”
published


publishing
“Southern_Messenger”_under
garb
real_name

order
might_certainly_regarded

name
affixed
articles
table
contents
narrative
prove
nature
carry
them_sufficient
fear
score
will_seen
much
fact_misrepresented
first
pages
were_written
will_unnecessary
point
portion_ends
commences

difference
point

attorney
speculated
successfully
stocks


means_managed
property

school

gentleman
eccentric_manners--he_well_known
almost_every_person
employ
Lloyd
Barnard_also
well_known

nearly_years_older
always_talking
adventures

Island

places_visited

worth

half-deck
longer
night
party

Augustus
myself_were



false_modelling
frame

main_object
Vessels
gale
wind_laid-to
according
vessels_have_sails
made_Foretopsails
often_found
answer
purpose_better
species
Grampus_generally_laid-to

brought_diagonally_across

bows_point_within
degrees
direction

windward
course_receives
shock

situation
good_vessel_will_ride
heavy_gale
wind_without_shipping
drop

without
attention_being_requisite
part

except
account
noise_makes

rudder
effect_upon
vessel

rudder
torn
heavy_seas
room
helm

life

violence

tear
sail
pieces_feat
requires
perfect_hurricane
accomplish

Some_vessels_will_lie-to

trusted

return
axes
fully_convinced
crew_were
well
watch
taken
surprise

breaking

pitched
violently
attempt
superstitious_terrors
guilty_conscience
remembered
died
having_been_attacked_days
spasms
drinking
spirits
keeping
points

corpse_presented
minutes
death
like
been_drowned
lain
water
diagonally_across
completely_covering
band

allow
usual_rites
Having_given

avoid
sight


design_abandoned
left
washed

furious_lu

time
they_were_first
resemblance

camel

root
neck_they_carry
them
constant_supply

much
three_gallons
perfectly_sweet
fresh_water_have_been_found

great_quantity
being_usually_found
hillsides_near
shore_wherever

been
means
preserving
lives
thousands
seamen_employed
whale-fishery
pursuits

good_fortune
bring
storeroom
weighing_probably_sixty-five
having
quart
limpid
sweet_water
This_indeed
treasure
falling
knees
returned_fervent_thanks
seasonable
great_difficulty
getting
animal

struggles_were_fierce
point
making_escape

slipping_back

throwing
rope
slipknot_around_side

assisted
water_drew_carefully
jug
been_brought
broke
neck
bottle


kind
holding
quite_half

long

having_supped_plentifully
olives

small_allowance
secured_them
well
possible
cordage
fragments

were_anxious
preserve_alive
long
threw

endured
little
ordinary_evil--so_strictly_comparative_either_good

sheet_used

means
collecting
rain
holding
sheet_spread
forechain-plates
middle
drained

have_been_imagined_possible
noon

were
Under

height
five
built_among
forked_limbs
trees


bend
upon
thus_forming_thicker_shelter
consisted
apparently_scratched
face
precipitous_ledge

three_sides
primitive_caverns

tenant_carefully_placed

stone_itself_never
sufficient_size
close
third
Through
middle
valley
brawling_stream
magical-looking_water
largest
creatures_resembled_common
structure
body
snout


legs_slender
awkward

never_attempt
noticed_also_several_animals
similar

greater_length

covered

seemed
constitute
chief_food


always_returning
village

using
southern_shore
vicinity
place
followed_them
dwellings


fish
seemed

innumerable

latitude
Gallipago_tortoise_also


none

species

serpents


concluded
they_were

distinguish



weapons
country_seemed_also
possession

appearance
among
altogether_wanting


grace
freedom
carriage
found

were_thick

even

Their_hair
finer_texture

like
party

dresses

armed
lances
These_appeared
have_great_influence_among

were_always_addressed
were
tenants
That
Too-wit_situated
c


death
have_come_without_terrors
--but
desire
--for_life--but
life--solace
reason_were
placidity
voice_grew
gentle--grew
mortal--to_assumptions
aspirations
mortality_never

bosom
love
have_reigned_strength
pour
overflowing
heart_whose
passionate
beloved
hour
upon
subject_womanly_abandonment
wildly_earnest
desire
life
fleeing

wild_longing--it
eager_vehemence
desire_capable

gala_night_Within
While
orchestra_breathes_fitfully_music
Flapping
sure_shall
much
Madness
Horror
soul
amid

seraphs
Out--out
affirm_That
play
”_hero
shall
Conqueror

part
parcel
doth
yield

save
weakness

exhausted
suffered_white_arms

returned_solemnly

passage
Glanvill--“Man_doth
yield

save
weakness
longer_endure
lonely_desolation
dwelling

ordinarily_falls
utter_abandonment
driven
remote
unsocial_region

display
cornices

Bedlam_patterns
carpets
tufted_labors
orders_taken
coloring
Ligeia--the_fair-haired

There
individual_portion
architecture
decoration
bridal_chamber
visibly
they_permitted
pass
threshold
apart


Overhead_suspended

means
rope_tied_round
legs
fastened
ring
causing
whole_loose
rattling_frame
dangle
twirl
caprice
every_occasional_puff
wind
found

preventing
escaping
seamen
conduct_themselves
degree
decorum
burst

obstreperous_roar
ill-timed


seat
others
company_placed
meantime

pouring
skull
quaffed
becomes_duty_upon
rigged
like

swilling
snug_blue_ruin_stowed_away
winter
Will_Wimble
wild_fiendish_shrieks
caught
attention
Most_willingly_will_gratify
reasonable_curiosity
part
guests
unbidden_though_they_title
‘King_Pest
Will_Wimble
undertaker--a

whose_plebeian_appellation_never
night_thwarted_royal_ears--this_councils

sacred
noble_lady


might_pardoned
replying


private

manner_important

doing
advance
designs
true_welfare
unearthly_sovereign_whose_reign

helping
lady
side
skull

pouring
second
said
turning_attention
“profane
consideration
even
feel
inclination
have_condescended
make_reply
rude
knees--ye_shall_forthwith_free_either

remain
admitted
privileges
according
your_r

In [64]:
# Phrase frequency count
from operator import itemgetter, attrgetter
wordfreq = []
for u in range(len(phrases)):
    utterance = phrases[u-1]
    uttcnt = 0
    uttcnt = phrases.count(utterance)
    if uttcnt > 1:
        wordfreq.append(uttcnt)
    
zipped = list(zip(phrases, wordfreq))
sortzip = sorted(zipped, key=itemgetter(1), reverse=True)

for v in range(len(sortzip)):
    print(sortzip[v])

('extraordinary_series', 2656)
('account_given', 2656)
('society', 2656)
('give_narrative', 2656)
('myself', 2656)
('much_unavoidable_exaggeration', 2656)
('prone', 2656)
('detailing_events', 2656)
('exciting', 2656)
('related', 2656)
('published', 2656)
('', 2656)
('', 2656)
('publishing', 2656)
('“Southern_Messenger”_under', 2656)
('order', 2656)
('might_certainly_regarded', 2656)
('', 2656)
('name', 2656)
('affixed', 2656)
('narrative', 2656)
('nature', 2656)
('them_sufficient', 2656)
('will_seen', 2656)
('fact_misrepresented', 2656)
('first', 2656)
('pages', 2656)
('portion_ends', 2656)
('successfully', 2656)
('', 2656)
('', 2656)
('property', 2656)
('school', 2656)
('gentleman', 2656)
('eccentric_manners--he_well_known', 2656)
('employ', 2656)
('Lloyd', 2656)
('nearly_years_older', 2656)
('Island', 2656)
('', 2656)
('places_visited', 2656)
('', 2656)
('party', 2656)
('', 2656)
('never', 2656)
('sober', 2656)
('determined', 2656)
('delightful', 2656)
('', 2656)
('quite', 2656)
('ti

('foot_high', 2656)
('This_made', 2656)
('', 2656)
('', 2656)
('upon', 2656)
('return', 2656)
('partner', 2656)
('nestling', 2656)
('', 2656)
('thieving_propensities_prevalent', 2656)
('inhabitants_making', 2656)
('scruple', 2656)
('enjoying', 2656)
('', 2656)
('scattering_their_nests', 2656)
('', 2656)
('stations', 2656)
('', 2656)
('continually_hovering', 2656)
('marching', 2656)
('military_strut', 2656)
('around', 2656)
('encircles', 2656)
('elicit_reflection', 2656)
('', 2656)
('morning', 2656)
('Christmas_Harbour', 2656)
('point', 2656)
('shore_toward', 2656)
('design', 2656)
('leave', 2656)
('letter', 2656)
('height', 2656)
('vessel', 2656)
('expected', 2656)
('Peters', 2656)
('', 2656)
('cruise_around', 2656)
('looking', 2656)
('', 2656)
('', 2656)
('several_small_islands', 2656)
('were', 2656)
('crowned', 2656)
('', 2656)
('', 2656)
('procure_three_hundred', 2656)
('fifty_skins', 2656)
('', 2656)
('', 2656)
('smaller_islands_discovered', 2656)
('', 2656)
('gave', 2656)
('repres

('', 5)
('arrival', 5)
('Englishman', 5)
('group', 5)
('', 5)
('toward', 5)
('course', 5)
('near_vicinity', 5)
('', 5)
('', 5)
('point', 5)
('', 5)
('southeast', 5)
('course', 5)
('passing', 5)
('thermometer_being', 5)
('upon', 5)
('although', 5)
('masthead_gave', 5)
('were_soon_close', 5)
('', 5)
('effect', 5)
('fifty_fathoms', 5)
('albatross_passed', 5)
('look', 5)
('seemed', 5)
('repeating', 5)
('they_appeared', 5)
('hurting', 5)
('Too-wit', 5)
('', 5)
('articles', 4)
('fear', 4)
('', 4)
('', 4)
('frolic', 4)
('like', 4)
('', 4)
('', 4)
('hour_maintained', 4)
('knees_shook', 4)
('shipping_heavy_seas_occasionally', 4)
('chance', 4)
('excited_alternate_laughter', 4)
('', 4)
('', 4)
('help', 4)
('small_jolly_they_were', 4)
('have_escaped_destruction', 4)
('Augustus_struggling_near_apparently', 4)
('', 4)
('degree_clouded', 4)
('period', 4)
('deck_drifting_rapidly', 4)
('discovered', 4)
('severest_gales_ever_experienced', 4)
('ardent_longing', 4)
('within', 4)
('', 4)
('captivity_among_

# TF-IDF

https://cdn-images-1.medium.com/max/800/1*nSqHXwOIJ2fa_EFLTh5KYw.png

In [ ]:
# Computes ratio of word's appearances to total words
def computeTF(wordDict, bow): 
    bowCount = len(bow) #BOW = Bag of Words
    tfDict = {}
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

In [ ]:
# Calculates the weight of rare words across all docs
def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
                
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
    
    return idfDict

In [ ]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf